In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [2]:
df = pd.read_csv("salary_data_cleaned.csv", on_bad_lines='skip', engine='python')

In [3]:
features = [
    'Job Title', 'Job Description', 'Rating', 'Industry', 'Sector', 'Size',
    'Founded', 'python_yn', 'R_yn', 'spark', 'aws', 'excel'
]
target = 'avg_salary'

In [4]:
X = df[features]
y = df[target]

In [5]:
preprocessor = ColumnTransformer(transformers=[
    ('text', TfidfVectorizer(max_features=300), 'Job Description'),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['Job Title', 'Industry', 'Sector', 'Size']),
    ('num', 'passthrough', ['Rating', 'Founded', 'python_yn', 'R_yn', 'spark', 'aws', 'excel'])
])

In [6]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(max_features=300),
                                                  'Job Description'),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Job Title', 'Industry',
                                                   'Sector', 'Size']),
                                                 ('num', 'passthrough',
                                                  ['Rating', 'Founded',
                                                   'python_yn', 'R_yn', 'spark',
                                                   'aws', 'excel'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [9]:
y_pred = model_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [10]:
print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

# Save the model
joblib.dump(model_pipeline, "salary_predictor_model.pkl")
print("Model saved to salary_predictor_model.pkl")

Mean Squared Error: 502.27552332214765
R² Score: 0.6927059750860227
Model saved to salary_predictor_model.pkl
